# **openai api , faiss를 부른다**

In [1]:
pip install faiss-cpu openai # faiss는 openai 호출 전 rag활용 목적

In [2]:
import openai
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer # sentence_transformers로 단어간이 아니라 문장 간의 유사성을 벡터공간에서 찾는다
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
openai.api_key = "Your api key"

# **문서 임베딩 공간 생성**

document

In [4]:
#문서 리스트 정의 (임베딩할 텍스트 데이터)
documents = [
    "..."
]

In [5]:
# 사전 학습된 임베딩 모델 로드 (예: all-MiniLM-L6-v2)
embedding_model = SentenceTransformer("embedding_model")
doc_embeddings = embedding_model.encode(documents)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# 2. FAISS 인덱스 구축
index = faiss.IndexFlatL2(doc_embeddings.shape[1])  # L2 거리 사용
index.add(np.array(doc_embeddings))  # 임베딩 추가

In [7]:
# 3. 질문을 통한 관련 문서 검색
def retrieve_relevant_documents(query, top_k=2):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), top_k)
    return [documents[i] for i in indices[0]]

## **ChatGPT API 호출**

In [8]:
# 4. ChatGPT API 호출
def generate_answer(query):
    # 질문에 관련된 문서 검색
    related_docs = retrieve_relevant_documents(query)
    context = "\n".join(related_docs)  # 검색된 문서들을 하나의 텍스트로 결합

    # ChatGPT API 호출
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": f"Context: {context}\nQuestion: {query}"},
        ],
        max_tokens=1000,
        temperature=0.7
    )

    return response.choices[0].message["content"]

# **출력 결과**

질문

In [9]:
query = "What can Python be used for?"

# **generate_answer에 오류가 생기므로 openai 버전을 downgrade시킨다**

In [10]:
!pip install openai==0.28

In [11]:
#질문에 대한 답변 생성
answer = generate_answer(query)
print("Answer:", answer)

Answer: Python can be used for a wide variety of applications, including but not limited to:

1. **Web Development**: Python can be used to build web applications using frameworks like Django and Flask.

2. **Data Analysis and Visualization**: Libraries such as Pandas, NumPy, and Matplotlib allow for data manipulation and visualization, making Python popular in data science.

3. **Machine Learning and AI**: Python is widely used for machine learning and artificial intelligence development with libraries like TensorFlow, Keras, and Scikit-learn.

4. **Automation and Scripting**: Python is often used to automate repetitive tasks, such as file manipulation, web scraping, and data entry.

5. **Game Development**: Libraries like Pygame allow for the creation of games and multimedia applications.

6. **Internet of Things (IoT)**: Python can be used to program IoT devices, including smart home devices such as smart air conditioners, leveraging libraries for hardware interaction.

7. **Web Scr